# Surface Event Pick Time

This is a modified version of the surface-event location+directivity analysis that Francesca Skene (fskene@uw.edu), originally created by her in 7/22/22, who started as an undergraduate student at UW. This is marine denolle's version. It includes:
* Waveform download for each event on each volcano given the PNSN pick times of "su" events.
* Data pre-processing to trim the data within 2-12 Hz and remove outliers.
* phase picking using transfer-learned model (Ni et al, 2023)
* event location using 1D grid search
* directivity measurements (velocity and direction) using Doppler effects.
* gathering of the data into a CSV data frame.

Here we will try to locate the 04/09/2020 avalanche on Carbon glacier headwall for its 4th year anniversary!

Updated 04/9/2024
Marine Denolle
(mdenolle@uw.edu)

Import Modules

In [ ]:
# import sys
# sys.path.append('/data/wsd01/pnwstore/')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import obspy
from obspy.core import UTCDateTime
from obspy.clients.fdsn.client import Client

import scipy
from scipy import optimize
from scipy.optimize import curve_fit
from datetime import datetime
from utils import *
from mbf_elep_func import *
import torch
plt.rcParams.update({'font.size': 10})


import seisbench.models as sbm
device = torch.device("cpu") #torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# from ELEP.elep.ensemble_statistics import ensemble_statistics
from ELEP.elep.ensemble_coherence import ensemble_semblance 
# from ELEP.elep.ensemble_learners import ensemble_regressor_cnn
from ELEP.elep import mbf, mbf_utils
from ELEP.elep import trigger_func

from ELEP.elep.mbf_utils import make_LogFq, make_LinFq, rec_filter_coeff, create_obspy_trace
from ELEP.elep.mbf import MB_filter as MBF

from joblib import Parallel, delayed
from matplotlib.backends.backend_pdf import PdfPages
import pyproj

What avalanche are we studying?

In [ ]:
ava = 'Avalanche_04092020'

Parameters

In [ ]:
# define clients to download the station data
# client = WaveformClient() # we ignore PNWdatastore for now
client2 = Client('IRIS') # IRIS client

t_before = 15 #number of seconds before pick time
# t_after = 15 #number of seconds after pick time
t_before_raw = 1200 #number of seconds before pick time before removing instrumental response
# t_after_raw = 1200 #number of seconds after pick time before removing instrumental response
fs = 40 #sampling rate that all waveforms are resampled to
window = 150 #window length of the signal (this will help with phase picking with EqT next). 
# Use 150 seconds @ 40 Hz gives 6001 points. 
pr = 98 #percentile
thr = 7 #SNR threshold
station_distance_threshold = 25
pi = np.pi
vs = 3000 #shear wave velocity at the surface

# range of dates that we are looking at
t_beginning = UTCDateTime(2001,1,1,0,0,0) 
t_end = UTCDateTime(2024,1,1,23,59)

smooth_length = 20 # constant for smoothing the waveform envelopes
low_cut = 1 #low frequency threshold
high_cut = 15 #high frequency threshold
az_thr = 1000 #threshold of distance in meters from source location
step = 100 #step every 100 m
t_step = 1 #step every second
ratio = 5.6915196 #used to define the grid 
# colors = list(plt.cm.tab10(np.arange(10)))*3
radius = 6371e3 # radius of the earth

## Volcano - Station Information

In [ ]:
#this data includes all stations within 50km of each volcano and the lat, lon, elev of each station
df = pd.read_csv('../data/station/Volcano_Metadata_50km.csv')

## PNSN SU Pick information

In [ ]:
# f1 = pd.read_csv("../data/events/su_picks.txt",sep="|")
# f1.head()
# print(f1.keys())

In [ ]:
# # clean up the spaces in the file
# format='%Y/%m/%d %H:%M:%S'
# test=f1["date"].values.tolist()
# start_time_temp = [  datetime.strptime(x.strip(),'%Y/%m/%d %H:%M:%S') for x in f1["date"].values.tolist()]
# # # Ignore events prior to t_beginning
# ik=np.where(np.array(start_time_temp)>datetime(2001,1,1))[0][0]

# # select only net, sta, evid, startime for event past the start date.

# start_time = start_time_temp[ik:]
# net=[ x.strip() for x in f1["net"].values.tolist()][ik:]
# sta=[ x.strip() for x in f1["sta"].values.tolist()][ik:]
# evt_id=[ x for x in f1["orid"].values.tolist()][ik:]
# all_stas=set(sta)

## ML Models

In [ ]:
# import os
# os.makedirs("/Users/marinedenolle/.seisbench/models/v3/eqtransformer",exist_ok=True)

In [ ]:
# !wget https://github.com/congcy/ELEP/raw/main/docs/tutorials/data/pnw.pt.v1 -O ~/.seisbench/models/v3/eqtransformer/pnw.pt.v1
# !wget https://github.com/congcy/ELEP/raw/main/docs/tutorials/data/pnw.json.v1 -O ~/.seisbench/models/v3/eqtransformer/pnw.json.v1

In [ ]:
# download models
list_models_name = ["pnw","ethz","instance","scedc","stead","geofon"]
pn_pnw_model = sbm.EQTransformer.from_pretrained('pnw')
pn_ethz_model = sbm.EQTransformer.from_pretrained("ethz")
pn_instance_model = sbm.EQTransformer.from_pretrained("instance")
pn_scedc_model = sbm.EQTransformer.from_pretrained("scedc")
pn_stead_model = sbm.EQTransformer.from_pretrained("stead")
pn_geofon_model = sbm.EQTransformer.from_pretrained("geofon")
# pn_neic_model = sbm.EQTransformer.from_pretrained("neic")

list_models = [pn_pnw_model, pn_ethz_model, pn_instance_model, pn_scedc_model, pn_stead_model, pn_geofon_model]

pn_pnw_model.to(device);
pn_ethz_model.to(device);
pn_scedc_model.to(device);
# pn_neic_model.to(device);
pn_geofon_model.to(device);
pn_stead_model.to(device);
pn_instance_model.to(device);

In [ ]:
paras_semblance = {'dt':0.025, 'semblance_order':4, 'window_flag':True, 
                   'semblance_win':0.5, 'weight_flag':'max'}
p_thrd, s_thrd = 0.01, 0.05

fqmin = low_cut
fqmax = high_cut
dt = 0.025; fs = 40
nfqs = 10
nt = 6000; nc = 3
fq_list = make_LogFq(fqmin, fqmax, dt, nfqs)
coeff_HP, coeff_LP = rec_filter_coeff(fq_list, dt)
MBF_paras = {'f_min':fqmin, 'f_max':fqmax, 'nfqs':nfqs, 'frequencies':fq_list, 'CN_HP':coeff_HP, 'CN_LP':coeff_LP, \
    'dt':dt, 'fs':fs, 'nt':nt, 'nc':nc, 'npoles': 2}

# Full stack:

* download waveforms
* phase pick onset
* estimate SNR
* measure centroid, max envelope, duration
* measure Fmax for doppler analysis


In [ ]:

pdf = PdfPages('../plots/'+ava+'.pdf')
associated_volcano = "Mt_Rainier"
dff=[] 
event_ID = '0000' #str(evt_id[n])
otime = UTCDateTime(2020,4,9,13,28,30)
associated_volcano="Mt_Rainier"


#get info for stations within 50km of volcano that event ocurred at
stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()


#################### WAVEFORM DOWNLOAD #######################
#Download all waveforms for that event based on stations and times
bulk = [] 
for m in range(0, len(networks)):
    if stations[m]=="PCEP" or stations[m]=="LON" or stations[m]=="LO2":continue
    bulk.append([networks[m], stations[m], '*', '*Z', otime-t_before_raw, otime+t_before_raw])
# try:
st = client2.get_waveforms_bulk(bulk)
st = resample(st,fs)  #resampling the data to 40Hz for each trace
evt_data = obspy.Stream()
snr=[]
stas=[]
nets=[]
lats=[]
lons=[]
els=[]
centroid_time = []
data_env_dict = {}
duration = []
max_time = []

# #Keeping all traces for one event with channel z, SNR>10, and bandpassed between 2-12Hz
# ,nets,max_amp_times,durations,data_env_dict,t_diff = [],[],[],[],[],[],[],{},{}
for i,ii in enumerate(st):
    ii.detrend(type = 'demean')
    ii.filter('bandpass',freqmin=low_cut,freqmax=high_cut,corners=2,zerophase=True)
    # trim the data and noise window to exactly 6000 points
    signal_window = ii.copy()
    noise_window = ii.copy()
    signal_window.trim(otime - t_before, otime - t_before + window) # trim the signal at the first pick time of the PNSN data, with loose 40s before
    noise_window.trim(otime - window -t_before, otime - t_before) # noise window of the same length
    if  len(signal_window.data)<=10 or  len(noise_window.data)<=10: continue # skip if no data
    # if not np.percentile(np.abs(signal_window.data),pr):continue # skip if max amplitude is zero
    snr1 = (20 * np.log(np.percentile(np.abs(signal_window.data),pr) 
                    / np.percentile(np.abs(noise_window.data),pr))/np.log(10))
    print("snr",snr1)
    if snr1<thr: # and 100<max_amp_time<200:
        st.remove(ii)
        continue

################# ENVELOPE, CENTROID, DURATION #######################
    # enveloping the data 
    data_envelope = obspy.signal.filter.envelope(signal_window.data)
    data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)

    data_env_dict[ii.stats.network+'.'+ii.stats.station]= data_envelope/max(np.abs(data_envelope))


    # max time
    # finding the time of max amplitude of each event
    # signal_window is windowed at otime-t_v before the PNSN pick time
    crap = np.argmax(np.abs(data_envelope[:(t_before+40)*fs])) # time of max amplitude relative to otime
    max_time.append(crap/fs)

    # centroid time
    tcrap = signal_window.times()
    it = np.where(tcrap>0)[0]
    centroid_time.append(np.sum(data_envelope*tcrap)/np.sum(data_envelope))
    print(ii.stats.station,max_time[-1],centroid_time[-1])

    # find duration as data starting with the "origin time" and ending when the envelope falls below the mean noise
    noise_envelope = obspy.signal.filter.envelope(noise_window.data)
    data_envelope = obspy.signal.util.smooth(data_envelope, smooth_length)
    mean_noise = np.mean(noise_envelope)
    
    mmax = np.max(np.cumsum(data_envelope**4))
    crap = np.where( np.cumsum(data_envelope**4) <= 0.999*mmax)[0][-1]
    duration.append(crap/fs)


    stas.append(ii.stats.station)
    nets.append(ii.stats.network)
    ista=stations.index(ii.stats.station)
    lats.append(latitudes[ista])
    lons.append(longitudes[ista])
    els.append(elevations[ista])
    snr.append(snr1)
    evt_data.append(signal_window)

    t = evt_data.select(station=stas[-1])[0].times()
    

centroid_time = np.asarray(centroid_time)
centroid_time -= t_before
max_time = np.asarray(max_time)
max_time -= t_before
duration = np.asarray(duration)
duration -= t_before

################### ELEP #######################

    # test the new function
smb_peak= apply_elep(evt_data, stas, \
        list_models, MBF_paras, paras_semblance, t_before)
smb_peak -= t_before


############## PEAK FREQUENCY MEASUREMENTS ############
# Given the approximate measurement of duration, window the signal windows around that
# then measure peak frequency so that there is less noise in it.
# perform this on the Z component only.

char_freq, sharp_weight= [],[]
fig,ax = plt.subplots(1,1,figsize=(11,8), dpi = 200)
for ii,i in enumerate(evt_data):
    data = np.zeros(200*fs)
    crap=i.copy()
    otime1 = crap.stats.starttime + smb_peak[ii] # pick time
    crap.trim(otime1  - 10, otime1 + 1.5*duration[ii] + 10) # window the data around the pick time
    crap.taper(max_percentage=0.01,max_length=20)

    data[:len(crap.data)] = crap.data #*100
    f,psd=scipy.signal.welch(data,fs=fs,nperseg=81,noverlap=4)
    #just get the frequencies within the filter band
    above_low_cut = [f>low_cut]
    below_high_cut = [f<high_cut]
    in_band = np.logical_and(above_low_cut,below_high_cut)[0]
    f = f[in_band]
    psd = psd[in_band]

    # calculate characteristic frequency and report
    char_freq_max = f[np.argmax(psd)]
    char_freq_mean= np.sum(psd*f)/np.sum(psd)
    psd_cumsum = np.cumsum(psd)
    psd_sum = np.sum(psd)
    char_freq_median = f[np.argmin(np.abs(psd_cumsum-psd_sum/2))]
    char_freq.append(char_freq_mean)

    plt.rcParams.update({'font.size': 20})
    p=ax.plot(f,psd,label=stas[ii],linewidth=2)
    cc = p[0].get_color()
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid('True')
    ax.set_xlabel('Frequency [Hz]')
    ax.set_ylabel('PSD [$(mm/s)^2$/Hz]')
    ax.vlines(char_freq_mean,ymin=np.min(psd)/10,ymax=np.max(psd)*10,linestyle="--",colors=cc)
    ax.grid(True)

#             # weighting the data by the spikiness of the PSD vs frequency graphs
    ratio = (np.mean(psd)/np.max(psd))
    sharp_weight.append(int(1/(ratio**2)*20))


    ############# KEEP DATA #######################

#         if not max(smb_peak.shape):continue
ddict = {'otime':otime, 'nets':nets, 'stas':stas,  'snr':snr, 'smb_peak': smb_peak, 'max_time':max_time, 'centroid_time': centroid_time , \
         'lats':lats, 'lons':lons, 'elevs':els, 'char_freq':char_freq, 'duration':duration, \
            'sharp_weight':sharp_weight, 'volcano':associated_volcano, 'event_ID':event_ID}
if not np.any(dff):
    dff = pd.DataFrame.from_dict(ddict)
else:
    dff=pd.concat([dff,pd.DataFrame.from_dict(ddict)],ignore_index=True)
print(dff)



fig = plt.figure(figsize = (11,8), dpi=400)
fig.suptitle(str(otime)+" "+associated_volcano)
ax = plt.subplot(1,1,1)
iplot = 0
for i in range(len(stas)):
    data = evt_data.select(station=stas[i])[0].data
    max1 = np.max(np.abs(data))
    t = evt_data.select(station=stas[i])[0].times()
    ax.plot(t-t_before,data/max1+iplot*1.5,linewidth=0.5)
    if np.any(data_env_dict[nets[i]+'.'+stas[i]]):
        ax.plot(t-t_before,data_env_dict[nets[i]+'.'+stas[i]]+iplot*1.5,'k',linewidth=1)
    ax.plot(smb_peak[i],iplot*1.5,'r*',markersize=5)  
    ax.plot(centroid_time[i],iplot*1.5,'k*',markersize=5)  
    ax.plot(max_time[i],iplot*1.5,'r*',markersize=5)  
    ax.set_yticks([])
    plt.text(-15, iplot*1.5+0.5, stas[i])
    # print(stas[i],snr[i])
    # if i==ista:
    # err_title=("%s %2.2f (s) error in picks"%(stas[i],smb_peak[i]-t_before))
    # plt.text(60, iplot*1.5+0.5,err_title,color='r')
    plt.vlines(smb_peak[i],iplot*1.5-1.,iplot*1.5+1.,'r')
    # print(duration[i],char_freq[i],sharp_weight[i])
    # print(stas[i],smb_peak[i]-t_before)
    iplot+=1
# plt.grid(True)
ax.set_xlim([-t_before,130])
ax.set_xlabel('Time (s) since 13:28 ')
plt.show()
plt.savefig("../plots/" + ava + "_waveforms.png")
plt.clf()
# del fig
pdf.close()
dff.to_csv("../data/events/MLPicks_" + ava +".csv")

Now locate the event

## Volcano data



In [ ]:

associated_volcano == 'Mt_Rainier'
        
#get info for stations within 50km of volcano that event ocurred at
stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()

############ LOCATION ############################
# input necessary data for grid search
arrivals = dff['smb_peak'].values
# arrivals = dff[dff['event_ID']==event_ID]['smb_peak'].values
sta_lats = dff['lats'].values
sta_lons = dff['lons'].values


In [ ]:
import pyproj


# # Define the projection: UTM zone 11 for Washington state
# proj = pyproj.Proj(proj='utm', zone=11, ellps='WGS84')

# # Convert lat/long to Cartesian in meters
# xsta, ysta = proj(sta_lons, sta_lats)

# cmap = plt.get_cmap('hot_r')
# ik=np.where(arrivals>0)[0]
# for i in ik:
#     tt = arrivals[i]-np.min(arrivals[ik])
#     nmax=np.max(arrivals[ik])-np.min(arrivals[ik])
#     plt.plot(xsta[i],ysta[i],'o',color=cmap(tt/nmax),markersize=10,markeredgecolor='k')
#     plt.text(xsta[i],ysta[i],stas[i]+":"+str(np.ceil(tt))+" s")
#     plt.axis('equal')
# plt.title("Travel time Relative to RCM")

Now we are confident that we can do the grid search. let's check the other fields

In [ ]:
# center latitude, center longitude, elevation(m), left_trim, right_trim, bottom_trim, top_trim 
volc_lat_lon = {}
volc_lat_lon['Mt_Rainier'] = [46.8528857, -121.7603744, 4392.5]
#Find the lower left corner and grid size based on volcano elevation
# define grid origin in lat,lon and grid dimensions in m
lon_start = -121.8 #volc_lat_lon[associated_volcano][0]
lon_end = -121.5 #volc_lat_lon[associated_volcano][0]
lat_start = 46.7 #volc_lat_lon[associated_volcano][1]
lat_end = 47 #volc_lat_lon[associated_volcano][1]


In [ ]:
ista =np.where(arrivals>-t_before+0.1)[0]
t_best,lon_best,lat_best = gridsearch_parallel(lat_start,lon_start,\
                                              lat_end,lon_end,\
                                                sta_lats[ista],sta_lons[ista],arrivals[ista],vs=vs,
                                                weight=dff['snr'].values[ista]**2)

print(t_best,lon_best,lat_best)


Now let's test this simple grid search

In [ ]:
ista =np.where(arrivals>-t_before+0.1)[0]
rss_mat,t_best,lon_best,lat_best,best_idx = gridsearch(lat_start,lon_start,\
                                              lat_end,lon_end,\
                                                sta_lats[ista],sta_lons[ista],arrivals[ista],vs=vs,
                                                weight=dff['snr'].values[ista]**2)

# lets plot rss_mat to see if we can find the minimum
    
    

Now also find the location of the first max time

In [ ]:
vs=2000

In [ ]:
ista =np.where(arrivals>-t_before+0.1)[0]
t_best,mlon_best,mlat_best = gridsearch_parallel(lat_start,lon_start,\
                                              lat_end,lon_end,\
                                                sta_lats[ista],sta_lons[ista],dff['max_time'].values[ista],vs=vs)
print(mlon_best,mlat_best)

In [ ]:
ista =np.where(arrivals>-t_before+0.1)[0]
t_best,clon_best,clat_best = gridsearch_parallel(lat_start,lon_start,\
                                              lat_end,lon_end,\
                                                sta_lats[ista],sta_lons[ista],dff['centroid_time'].values[ista],vs=vs)
print(clon_best,clat_best)

Now we are going to figure out what is going on with earthquake location.

1. Grid search

In [ ]:
vs=500
ista =np.where(arrivals>-t_before+0.1)[0]
rss_mat,t_best,lon_best,lat_best,best_idx = gridsearch(lat_start,lon_start,\
                                              lat_end,lon_end,\
                                                sta_lats[ista],sta_lons[ista],arrivals[ista],vs=vs,
                                                weight=dff['snr'].values[ista]**2)

## Solve for the best velocity

In [ ]:
vs=500
ista =np.where(arrivals>-t_before+0.1)[0] 
t_best,lon_best,lat_best,v_best = gridsearch_parallel_vs(lat_start,lon_start,\
                                              lat_end,lon_end,\
                                                sta_lats[ista],sta_lons[ista],arrivals[ista],
                                                weight=dff['snr'].values[ista]**2)

print(t_best,lon_best,lat_best,v_best) 

Plot the residual curve

In [ ]:
# lets plot rss_mat to see if we can find the minimum
fig, ax = plt.subplots(figsize=(16, 10), dpi=400)
im = plt.imshow(np.squeeze(rss_mat[0,:,:].T),extent=[lon_start,lon_end,lat_start,lat_end], cmap='hsv', interpolation='nearest',origin='lower')
# Add contour lines
num_contour_lines = 100  # Change this to the number of contour lines you want
contours = plt.contour(np.squeeze(rss_mat[0,:,:].T), num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
plt.clabel(contours, inline=True, fontsize=8)
plt.colorbar(im)
plt.title("RSS for Mt. Rainier")


#### Create synthetics and plot them

In [ ]:
# given the source location, calculate travel times
proj = pyproj.Proj(proj='utm', zone=10, ellps='WGS84')

if lon_start<0: 
    lon_start1 = lon_start+360
    lon_end1 = lon_end + 360

# Convert lat/long to Cartesian in meters
x_step=100
x1,y1=proj(lon_start,lat_start)
x2,y2=proj(lon_end,lat_end)
su_x,su_y=proj(lon_best,lat_best)
# Generate the x and y coordinates for the grid
x_coords = np.arange(x1, x2, x_step)
y_coords = np.arange(y1, y2, x_step)
synthetic_tt = np.zeros((len(x_coords),len(y_coords)))    
for i in range(len(x_coords)):
    for j in range(len(y_coords)):
        synthetic_tt[i,j] = np.sqrt( (x_coords[i] - su_x  )**2 + (y_coords[j] - su_y)**2  ) / vs 
    
# Existing imshow plot
fig, ax = plt.subplots(figsize=(16, 10), dpi=400)
im = plt.imshow(np.squeeze(rss_mat[0,:,:].T),extent=[lon_start,lon_end,lat_start,lat_end], cmap='hsv', interpolation='nearest',origin='lower')


# Add contour lines
num_contour_lines = 20  # Change this to the number of contour lines you want

contours = plt.contour(synthetic_tt.T, num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
# contours = plt.contour(np.squeeze(rss_mat[0,:,:].T), num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
# contours = plt.contour(synthetic_tt, num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
plt.clabel(contours, inline=True, fontsize=8)
# plt.plot(bootstrap_lon_best,bootstrap_lat_best,'o',color='k',markersize=10)
cmap1 = plt.get_cmap('hot_r')
cmap2 = plt.get_cmap('hsv')
ik=np.where(arrivals>0)[0]
for i in ik:
    tt = arrivals[i]-np.min(arrivals[ik])
    nmax=np.max(arrivals[ik])-np.min(arrivals[ik])
    plt.plot(sta_lons[i],sta_lats[i],'o',color=cmap2(tt/10),markersize=dff[dff['stas']==stas[i]]['snr'].values,markeredgecolor='k')
    plt.text(sta_lons[i],sta_lats[i],stas[i]+":"+str(np.ceil(tt*10)/10)+" s")
plt.axis('equal')
plt.plot(lon_best,lat_best,'o',color='k',markersize=10)
plt.title("Travel time Relative to STAR, marker size shows SNR of waveform")
plt.xlim([-122,-121.5]);
plt.savefig("../plots/" + ava+ "_res.png")



## Plot

In [ ]:
# Existing imshow plot
fig, ax = plt.subplots(figsize=(16, 10), dpi=400)
im = plt.imshow(np.squeeze(rss_mat[0,:,:].T),extent=[lon_start,lon_end,lat_start,lat_end], cmap='hsv', interpolation='nearest',origin='lower')

# Add contour lines
num_contour_lines = 20  # Change this to the number of contour lines you want
contours = plt.contour(np.squeeze(rss_mat[0,:,:].T), num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
plt.clabel(contours, inline=True, fontsize=8)
plt.plot(lon_best,lat_best,'o',color='k',markersize=10)
plt.plot(mlon_best,mlat_best,'o',color='r',markersize=10)
plt.plot(clon_best,clat_best,'o',color='r',markersize=10)
cmap1 = plt.get_cmap('hot_r')
ik=np.where(arrivals>0)[0]
for i in ik:
    tt = arrivals[i]-np.min(arrivals[ik])
    nmax=np.max(arrivals[ik])-np.min(arrivals[ik])
    plt.plot(sta_lons[i],sta_lats[i],'o',color=cmap1(tt/nmax),markersize=20,markeredgecolor='k')
    plt.text(sta_lons[i],sta_lats[i],stas[i]+":"+str(np.ceil(tt))+" s")
plt.title("Travel time Relative to STAR")
plt.xlim([-122,-121.5]);



ok, the location works but seems quite innacurate: why is the location not closer to the first stations that saw it?
We will do a bootstrap grid search and find an enemble of solution and take the median location.

In [ ]:
# ista =np.where(arrivals>-t_before+0.1)[0]
# # Number of bootstrap samples
# n_bootstrap = 100

# # Initialize an array to hold the bootstrap results
# bootstrap_results = []
# llon_best=np.zeros(n_bootstrap)
# llat_best=np.zeros(n_bootstrap)
# for ii in range(n_bootstrap):
#     # Generate a bootstrap sample from ista
#     ista_sample = np.random.choice(ista, size=len(ista), replace=True)
#     # ista_sample=ista
#     # Perform the grid search with the bootstrap sample
#     t_best, lon_best, lat_best = gridsearch_parallel(lat_start, lon_start, lat_end, lon_end,\
#                                                                 sta_lats[ista_sample], sta_lons[ista_sample],\
#                                                                     arrivals[ista_sample])

#     # Store the results
#     llon_best[ii] = lon_best
#     llat_best[ii] = lat_best
#     print(llat_best[ii],llon_best[ii])
#     break
# bootstrap_lon_best=np.mean(llon_best)
# bootstrap_lat_best=np.mean(llat_best)
    

After some experimentation, we find that the locations found by the grid search are highly dependent on the choice of stations used. Weighting with the SNR brings even more variability.

In [ ]:
# # Existing imshow plot
# fig, ax = plt.subplots(figsize=(16, 10), dpi=400)
# im = plt.imshow(np.squeeze(rss_mat[0,:,:].T),extent=[lon_start,lon_end,lat_start,lat_end], cmap='hsv', interpolation='nearest',origin='lower')

# # Add contour lines
# num_contour_lines = 20  # Change this to the number of contour lines you want
# contours = plt.contour(np.squeeze(rss_mat[0,:,:].T), num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
# plt.clabel(contours, inline=True, fontsize=8)
# plt.plot(llon_best,llat_best,'o',color='k',markersize=6)
# plt.plot(bootstrap_lon_best,bootstrap_lat_best,'o',color='k',markersize=10)
# cmap1 = plt.get_cmap('hot_r')
# ik=np.where(arrivals>0)[0]
# for i in ik:
#     tt = arrivals[i]-np.min(arrivals[ik])
#     nmax=np.max(arrivals[ik])-np.min(arrivals[ik])
#     plt.plot(sta_lons[i],sta_lats[i],'o',color=cmap1(tt/nmax),markersize=dff[dff['stas']==stas[i]]['snr'].values,markeredgecolor='k')
#     plt.text(sta_lons[i],sta_lats[i],stas[i]+":"+str(np.ceil(tt))+" s")
# plt.title("Travel time Relative to RCM, marker size shows SNR of waveform")
# plt.xlim([-122,-121.5]);



In [ ]:
# Existing imshow plot
fig, ax = plt.subplots(figsize=(16, 10), dpi=400)
im = plt.imshow(np.squeeze(rss_mat[0,:,:].T),extent=[lon_start,lon_end,lat_start,lat_end], cmap='hsv', interpolation='nearest',origin='lower')


# Add contour lines
num_contour_lines = 20  # Change this to the number of contour lines you want

contours = plt.contour(synthetic_tt.T, num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
# contours = plt.contour(np.squeeze(rss_mat[0,:,:].T), num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
# contours = plt.contour(synthetic_tt, num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
plt.clabel(contours, inline=True, fontsize=8)
# plt.plot(bootstrap_lon_best,bootstrap_lat_best,'o',color='k',markersize=10)
cmap1 = plt.get_cmap('hot_r')
cmap2 = plt.get_cmap('hsv')
ik=np.where(arrivals>0)[0]
for i in ik:
    tt = arrivals[i]-np.min(arrivals[ik])
    nmax=np.max(arrivals[ik])-np.min(arrivals[ik])
    plt.plot(sta_lons[i],sta_lats[i],'o',color=cmap2(tt/10),markersize=dff[dff['stas']==stas[i]]['snr'].values,markeredgecolor='k')
    plt.text(sta_lons[i],sta_lats[i],stas[i]+":"+str(np.ceil(tt*10)/10)+" s")
plt.axis('equal')
plt.plot(lon_best,lat_best,'o',color='k',markersize=10)
plt.title("Travel time Relative to STAR, marker size shows SNR of waveform")
plt.xlim([-122,-121.5]);
plt.savefig("../plots/" + ava+ "_res.png")



Now locate the centroid.

## DEM

We need to reproject the DEM onto our own grid (x_coord, y_coord)s

In [ ]:

import rasterio
from matplotlib.colors import LightSource
from rasterio.warp import transform_bounds,  reproject, Resampling , calculate_default_transform
from rasterio.transform import array_bounds

In [ ]:
associated_volcano = 'Mt_Rainier'
        
#get info for stations within 50km of volcano that event ocurred at
stations = df[df['Volcano_Name'] == associated_volcano]['Station'].values.tolist()
networks = df[df['Volcano_Name'] == associated_volcano]['Network'].values.tolist()
latitudes = df[df['Volcano_Name'] == associated_volcano]['Latitude'].values.tolist()
longitudes = df[df['Volcano_Name'] == associated_volcano]['Longitude'].values.tolist()
elevations = df[df['Volcano_Name']== associated_volcano]['Elevation'].values.tolist()


In [ ]:
# center latitude, center longitude, elevation(m), left_trim, right_trim, bottom_trim, top_trim 
volc_lat_lon = {}
volc_lat_lon['Mt_Rainier'] = [46.8528857, -121.7603744, 4392.5]
#Find the lower left corner and grid size based on volcano elevation
# define grid origin in lat,lon and grid dimensions in m
lon_start = -121.9 #volc_lat_lon[associated_volcano][0]
lon_end = -121.65 #volc_lat_lon[associated_volcano][0]
lat_start = 46.6 #volc_lat_lon[associated_volcano][1]
lat_end = 47 #volc_lat_lon[associated_volcano][1]


# Create a light source
ls = LightSource(azdeg=315, altdeg=45)


# Load the DEM with rasterio
with rasterio.open('../data/geospatial/Mt_Rainier/Mt_Rainier.tif') as src:
    dem = src.read(1)  # read the first band
    transform = src.transform
    bounds = src.bounds
    crs=src.crs
    # dem = dem.astype('float64')
    dem[dem == -32767] = np.nan #gets rid of edge effects
    # dem = np.nan_to_num(dem,nan=1000)


# Define the target CRS
dst_crs = 'EPSG:4326'  # EPSG:4326 is the code for WGS84 lat/lon

# Calculate the transform and dimensions for the reprojected DEM
transform_latlon, width, height = calculate_default_transform(crs, dst_crs, dem.shape[1], dem.shape[0], *bounds)


# Create an empty array for the reprojected DEM
dem_latlon = np.empty(shape=(height, width))

# Reproject the DEM
reproject(
    source=dem,
    destination=dem_latlon,
    src_transform=transform,
    src_crs=crs,
    dst_transform=transform_latlon,
    dst_crs=dst_crs,
    resampling=Resampling.nearest)

# Transform the bounds to the target CRS
left, bottom, right, top = transform_bounds(crs, dst_crs, *bounds)

# Calculate the illumination intensity
illumination = ls.hillshade(dem_latlon)
# new bounds
left, bottom, right, top = array_bounds(height, width, transform_latlon)

In [ ]:


fig, ax = plt.subplots(figsize=(16, 10), dpi=400)

# find the right UTM zone for the data
def get_utm_zone(longitude):
    return int((longitude + 180) / 6) + 1
utm_zone = get_utm_zone(np.mean(sta_lons))
# # Define the projection: UTM zone 11 for Washington state
proj = pyproj.Proj(proj='utm', zone=utm_zone, ellps='WGS84')
# Convert lat/long to Cartesian in meters
xsta, ysta = proj(sta_lons, sta_lats)
cmap = plt.get_cmap('hot_r')

plt.imshow(illumination, extent=[left, right, bottom,top ], cmap='gray',clim=[0,1],alpha=0.5,aspect='equal');
# plt.imshow(illumination, extent=[lon_start, lon_end, lat_start, lat_end], cmap='gray',\
    # clim=[0,1],alpha=0.5,aspect='equal')

issta=np.argmin(arrivals[ista])
stass=stas[ista[issta]]

# Plot the origin
plt.plot(lon_best,lat_best,'*',color='r',markersize=20,markeredgecolor='k')

plt.title(str(dff[dff['event_ID']==event_ID]['otime'].values[0])[0:13]+ " Travel time Relative to "+stass);

# im = plt.imshow(np.squeeze(rss_mat[0,:,:].T),extent=[lon_start,lon_end,lat_start,lat_end], cmap='hsv', interpolation='nearest',origin='lower')

# Add contour lines
num_contour_lines = 20  # Change this to the number of contour lines you want

# Add contour lines
num_contour_lines = 20  # Change this to the number of contour lines you want

contours = plt.contour(synthetic_tt.T, num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
# contours = plt.contour(np.squeeze(rss_mat[0,:,:].T), num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
# contours = plt.contour(synthetic_tt, num_contour_lines, extent=[lon_start,lon_end,lat_start,lat_end], colors='black')
plt.clabel(contours, inline=True, fontsize=8)
# plt.plot(bootstrap_lon_best,bootstrap_lat_best,'o',color='k',markersize=10)
cmap1 = plt.get_cmap('hot_r')
cmap2 = plt.get_cmap('hsv')
ik=np.where(arrivals>0)[0]
for i in ik:
    tt = arrivals[i]-np.min(arrivals[ik])
    nmax=np.max(arrivals[ik])-np.min(arrivals[ik])
    plt.plot(sta_lons[i],sta_lats[i],'o',color=cmap2(tt/10),markersize=dff[dff['stas']==stas[i]]['snr'].values,markeredgecolor='k')
    plt.text(sta_lons[i],sta_lats[i],stas[i]+":"+str(np.ceil(tt*10)/10)+" s")
plt.axis('equal')
# plt.plot(lon_best,lat_best,'o',color='k',markersize=10)
plt.title("Travel time Relative to RCM, marker size shows SNR of waveform")
plt.xlim([-122,-121.5]);
plt.savefig("../plots/" + ava + "_res_topo.png")
